# Assistants API 사용

새로운 [Assistants API](https://platform.openai.com/docs/assistants/overview)는 [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)의 발전된 형태로 보다 간단하게 assistant를 만들고 개발자가 Code Interpreter 및 Retrieval과 같은 강력한 도구에 접근할 수 있도록 하는 것을 목적으로 함

## Chat Completions API vs Assistants API

**Chat Completions API**의 기본 단위는 `Messages`이며, 여기에 `Model`(`gpt-3.5-turbo`, `gpt-4` 등)을 사용하여 `Completion`을 수행한다  
API는 가볍고 강력하지만 본질적으로 상태가 없기 때문에 대화 상태, 도구 정의, 검색 문서, 코드 실행을 수동으로 관리해야 함

**Assistants API**의 기본 단위

- 기본 모델, 지침, 도구, 문서(문맥)를 포함하는 `Assistants`,
- 대화의 상태를 나타내는 `Threads`,
- 텍스트 응답 및 다단계 도구 사용을 포함하여 `Thread`에서 `Assistant`의 실행을 구동하는 `Runs`

## Python SDK

In [43]:
!pip install --upgrade openai

In [44]:
!pip show openai | grep Version

Version: 1.75.0


## Pretty Printing Helper

In [1]:
import json

def show_json(obj):
  display(json.loads(obj.model_dump_json()))

## Complete Example with Assistants API

In [2]:
from google.colab import userdata
key = userdata.get('Openai')

In [3]:
import os

# !export OPENAI_API_KEY=
os.environ["OPENAI_API_KEY"] = key

In [4]:
from openai import OpenAI

client = OpenAI(api_key=key)

assistant = client.beta.assistants.create(
    name="Math Tutor2",
    instructions="너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘",
    model="gpt-4o",
)

show_json(assistant)

{'id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
 'created_at': 1744880007,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor2',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

대시보드를 통해 Assistant를 생성하든 API를 사용하든, Assistant ID를 추적하는 것이 중요하다.   
ID를 통해 Threads와 Runs 전반에 걸쳐 Assistant를 참조
  
  
다음으로, 새로운 Thread를 생성하고 그 안에 Message를 추가  
대화의 상태를 유지할 수 있어 매번 전체 메시지 기록을 다시 보낼 필요가 없음

## Threads

In [5]:
# 새로운 Thread 생성

thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_wza0rEwznSfRU6SV6GUHtKSE',
 'created_at': 1744880008,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [6]:
# thread에 message 추가

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role='user',
    content= "'방정식 3x + 11 = 14'를 풀어줘"
)

show_json(message)

{'id': 'msg_wR6pXxKsIkCxPesfexXRS7pX',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [], 'value': "'방정식 3x + 11 = 14'를 풀어줘"},
   'type': 'text'}],
 'created_at': 1744880010,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_wza0rEwznSfRU6SV6GUHtKSE'}

## Runs

Thread가 앞서 만든 Assistant와 연결되지 않았음  
Thread는 Assistants와 독립적으로 존재하며 이는 ChatGPT(모델/GPT에 thread가 연결되어 있는)를 사용해 본 사람들이 예상하는 것과 다를 수 있다


주어진 Thread에 대한 Assistant의 Completion을 얻으려면 Run을 생성해야 함  
Run을 생성하면 Assistant에게 Thread의 메시지를 살펴보고 조치를 취하라는 지시가 됨  
단일 응답을 추가하거나 도구를 사용할 수 있음


> Runs는 Assistants API와 Chat Completions API 사이의 주요 차이점입니다. Chat Completions에서는 모델이 단일 메시지로만 응답할 수 있는 반면, Assistants API에서는 Run을 통해 Assistant가 하나 또는 여러 도구를 사용하고 Thread에 여러 메시지를 추가할 수 있다

우리 Assistant에게 사용자에게 응답하도록 하려면 Run을 생성  
Assistant와 Thread를 모두 지정

In [7]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

show_json(run)

{'id': 'run_ZYbGLz9ryLxnTsU9lgX28qTz',
 'assistant_id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1744880010,
 'expires_at': 1744880610,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_wza0rEwznSfRU6SV6GUHtKSE',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': None,
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {},
 'reasoning_effort': None}

Chat Completions API에서 완성을 생성하는 것과 달리, **Run을 생성하는 것은 비동기 작업**   
이 작업은 Run의 메타데이터와 함께 즉시 반환되며 여기에는 처음에 `queued`로 설정된 `status`가 포함된다  
Assistant가 도구 사용과 메시지 추가와 같은 작업을 수행함에 따라 `status`가 업데이트된다


Assistant가 처리를 완료했는지 알기 위해 Run을 반복적으로 폴링할 수 있다   `queued` 또는 `in_progress` 상태만 확인하지만 실제로 Run은 사용자에게 표시할 수 있는 [다양한 상태 변경]을 겪을 수 있다

In [8]:
import time

def wait_on_run(run, thread):
  while run.status in ['queued', 'in_progress']:
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
    time.sleep(0.5)
  return run

In [9]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_ZYbGLz9ryLxnTsU9lgX28qTz',
 'assistant_id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
 'cancelled_at': None,
 'completed_at': 1744880013,
 'created_at': 1744880010,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1744880012,
 'status': 'completed',
 'thread_id': 'thread_wza0rEwznSfRU6SV6GUHtKSE',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 6,
  'prompt_tokens': 69,
  'total_tokens': 75,
  'prompt_token_details': {'cached_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0}},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {},
 'reasoning_effort': None}

## Messages

In [10]:
messages = client.beta.threads.messages.list(thread_id = thread.id)
show_json(messages)

{'data': [{'id': 'msg_1KI0zIRvHkJtk1Auu6RC0HDK',
   'assistant_id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [], 'value': 'x = 1'},
     'type': 'text'}],
   'created_at': 1744880012,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_ZYbGLz9ryLxnTsU9lgX28qTz',
   'status': None,
   'thread_id': 'thread_wza0rEwznSfRU6SV6GUHtKSE'},
  {'id': 'msg_wR6pXxKsIkCxPesfexXRS7pX',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "'방정식 3x + 11 = 14'를 풀어줘"},
     'type': 'text'}],
   'created_at': 1744880010,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'status': None,
   'thread_id': 'thread_wza0rEwznSfRU6SV6GUHtKSE'}],
 'has_more': False,
 

메시지들은 역시간 순서로 정렬되어 있다 - 이렇게 함으로써 가장 최근의 결과가 항상 첫 번째 '페이지'에 있게 된다(결과는 페이지네이션 될 수 있으므로)

 Chat Completions API의 메시지 순서와 반대이므로 주의


In [11]:
# 결과에 대한 추가적인 설명

message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="설명해 주시겠어요?"
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

wait_on_run(run, thread)

messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_QXfGDAJyAIrGFjpkeSHVeHLJ',
   'assistant_id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '3x + 11 = 14에서, 양변에서 11을 빼면 3x = 3이 됩니다. 양변을 3으로 나누면 x = 1이 나옵니다.'},
     'type': 'text'}],
   'created_at': 1744880017,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_ckhA0uODztywEUCpZckg4tif',
   'status': None,
   'thread_id': 'thread_wza0rEwznSfRU6SV6GUHtKSE'}],
 'has_more': False,
 'object': 'list',
 'first_id': 'msg_QXfGDAJyAIrGFjpkeSHVeHLJ',
 'last_id': 'msg_QXfGDAJyAIrGFjpkeSHVeHLJ'}

## Example

submit_message    
Thread에 Message를 생성한 다음, 새로운 Run을 시작하고 반환


get_response    
Thread의 메시지 목록을 반환

In [12]:
math_assistant_id = assistant.id

client= OpenAI()

def submit_message(assistand_id, thread, user_message):
  client.beta.threads.messages.create(
      thread_id = thread.id, role='user', content=user_message
  )
  return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

def get_response(thread):
  return client.beta.threads.messages.list(thread_id=thread.id, order='asc')

재사용할 수 있는 `create_thread_and_run` 함수를 정의했는데, 이는 실제로 API의 [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) 복합 함수와 거의 동일  
마지막으로 각각의 가상 사용자 요청을 새로운 Thread에 제출할 수 있다


이 모든 API 호출은 비동기 작업  
`asyncio`와 같은 비동기 라이브러리의 사용 없이도 실제로 코드에서 비동기 동작을 얻을 수 있음을 의미

In [13]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(math_assistant_id, thread, user_input)
    return thread, run


thread1, run1 = create_thread_and_run(
    "방정식 '3x + 11 = 14'를 풀어줘")
thread2, run2 = create_thread_and_run("선형대수에 대해 설명해줘")
thread3, run3 = create_thread_and_run("나는 수학을 싫어해. 어떻게 하면 좋을까?")

In [14]:
import time

def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))


run4 = submit_message(math_assistant_id, thread3, "고마워")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: 방정식 '3x + 11 = 14'를 풀어줘
assistant: x = 1

# Messages
user: 선형대수에 대해 설명해줘
assistant: 선형대수는 벡터와 행렬을 사용하여 선형 방정식과 변환을 연구하는 수학 분야입니다.

# Messages
user: 나는 수학을 싫어해. 어떻게 하면 좋을까?
assistant: 작은 성취감을 느낄 수 있는 간단한 문제부터 시작해보세요.

# Messages
user: 나는 수학을 싫어해. 어떻게 하면 좋을까?
assistant: 작은 성취감을 느낄 수 있는 간단한 문제부터 시작해보세요.
user: 고마워
assistant: 천만에요!



Assistant ID를 변경하기만 하면 어떤 새로운 Assistant에 대해서도 이 코드가 작동

# Tool

Assistants API의 핵심 기능 중 하나는 Code Interpreter, Retrieval 및 사용자 정의 Functions과 같은 도구를 우리의 Assistants에 장착할 수 있는 능력


In [15]:
assistant = client.beta.assistants.update(
    math_assistant_id,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
 'created_at': 1744880007,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor2',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}],
 'response_format': {'type': 'text'},
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []}, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

In [16]:
thread, run = create_thread_and_run(
    "코드를 사용하여 10개의 피보나치 숫자를 생성해줘"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: 코드를 사용하여 10개의 피보나치 숫자를 생성해줘
assistant: 첫 10개의 피보나치 숫자는 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]입니다.



## Step

Run은 하나 이상의 Steps로 구성된다  
각 Step에는 조회할 수 있는 `status`가 있다 사용자에게 Step의 진행 상황을 표시하는 데 유용

In [17]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [18]:
for step in run_steps.data:
  step_details = step.step_details
  print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_pWb0eBAnZDOlpxrpEwx04UB8',
   'code_interpreter': {'input': 'def fibonacci_sequence(n):\n    fib_sequence = [0, 1]\n    for i in range(2, n):\n        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])\n    return fib_sequence\n\nfibonacci_sequence(10)',
    'outputs': []},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_6vf73nY5vm6vdMRQHAx2B2CH'},
 'type': 'message_creation'}

null


1. `tool_calls` (하나의 Step에 하나 이상이 있을 수 있으므로 복수형)
2. `message_creation`

첫 번째 Step은 `tool_calls`로, `code_interpreter`를 사용
- 도구가 호출되기 전에 생성된 Python 코드인 `input`, Code Interpreter를 실행한 결과인 `output`


두 번째 Step은 `message_creation`, 사용자에게 결과를 전달하기 위해 Thread에 추가된 `message`가 포함

## file search


질문에 답할 때 assistant가 지식 기반으로 사용할 파일을 업로드하는 기능  
대시보드 또는 API에서 활성화할 수 있으며, 사용하고자 하는 파일을 업로드할 수 있다

In [22]:
vector_store = client.beta.vector_stores.create(name='machine learning')

file_paths = '/content/drive/MyDrive/fastcam/Part 9. AI API 활용/실습 자료/Chapter 3. OpenAI API/data/language_models_are_unsupervised_multitask_learners.pdf'
file_streams = [open(path, 'rb') for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id = vector_store.id,
    files=file_streams)

print(file_batch.status)
print(file_batch.file_counts)

AttributeError: 'Beta' object has no attribute 'vector_stores'

In [21]:
assistant = client.beta.assistants.update(
    math_assistant_id,
    tools=[{"type": "code_interpreter"}, {"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)
show_json(assistant)

NameError: name 'vector_store' is not defined

In [40]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

In [41]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "이름이 뭐야?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "가장 좋아하는 색이 뭐야?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["빨간색", "파랑색", "초록색", "노란색"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

이름이 뭐야?

가장 좋아하는 색이 뭐야?
0. 빨간색
1. 파랑색
2. 초록색
3. 노란색

Responses: ["I don't know.", 'a']


Assistant가 호출할 수 있도록 이 함수의 인터페이스를 JSON 형식으로 정의해

In [42]:
function_json = {
    "name": "display_quiz",
    "description": "학생에게 퀴즈를 표시하고 학생의 응답을 반환합니다. 하나의 퀴즈에 여러 개의 질문이 포함될 수 있습니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "제목과 옵션(객관식인 경우)이 있는 질문의 배열입니다.",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

대시보드나 API를 통해 Assistant를 업데이트

In [43]:
assistant = client.beta.assistants.update(
    math_assistant_id,
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
 'created_at': 1744880007,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor2',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'score_threshold': 0.0,
     'ranker': 'default_2024_08_21'}}},
  {'function': {'name': 'display_quiz',
    'description': '학생에게 퀴즈를 표시하고 학생의 응답을 반환합니다. 하나의 퀴즈에 여러 개의 질문이 포함될 수 있습니다.',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': '제목과 옵션(객관식인 경우)이 있는 질문의 배열입니다.',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type': 'array', 'items': {'type': 'string'}}},
       

퀴즈 요청

In [44]:
thread, run = create_thread_and_run(
    "두 가지 질문으로 퀴즈를 만들어줘. 하나는 주관식, 하나는 객관식으로, 그런 다음 답변에 대한 피드백을 보내줘"
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [45]:
show_json(run)

{'id': 'run_GlTAadBvKCl8ZDgC1O5QG3Mc',
 'assistant_id': 'asst_MzaKGofZz5LGUxVU837TcdXz',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1744880736,
 'expires_at': 1744881336,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_tKIA0SvrpV0YwVmAl83zWGZ8',
     'function': {'arguments': '{"title": "수학 퀴즈", "questions": [{"question_text": "2의 제곱은 무엇인가요?", "question_type": "FREE_RESPONSE"}, {"question_text": "3의 세제곱은 얼마인가요?", "question_type": "MULTIPLE_CHOICE", "choices": ["6", "9", "27", "81"]}]}',
      'name': 'display_quiz'},
     'type': 'function'},
    {'id': 'call_FV3GtkGifrqo5LOr2GQ4agpI',
     'function': {'arguments': '{"title": "퀴즈 피드백", "questions": [{"question_text": "주관식

required_action 필드는 도구가 우리에게 실행하고 그 결과를 Assistant에게 제출하도록 대기 중인 것을 나타냄 구체적으로   
display_quiz 함수



Assistant는 여러 도구를 호출하기로 선택할 수 있다

In [46]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': '수학 퀴즈',
 'questions': [{'question_text': '2의 제곱은 무엇인가요?',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': '3의 세제곱은 얼마인가요?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['6', '9', '27', '81']}]}

Assistant에서 제공한 인수를 사용하여 실제로`display_quiz 함수를 호출

In [47]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: 수학 퀴즈

2의 제곱은 무엇인가요?

3의 세제곱은 얼마인가요?
0. 6
1. 9
2. 27
3. 81

Responses: ["I don't know.", 'a']


응답을 얻었으므로 Assistant에게 다시 제출

이전에 분석한 `tool_call`에서 찾을 수 있는 `tool_call` ID가 필요  
응답들의 `list`를 `str`로 인코딩해야 함

In [48]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

BadRequestError: Error code: 400 - {'error': {'message': "Expected tool outputs for call_ids ['call_tKIA0SvrpV0YwVmAl83zWGZ8', 'call_FV3GtkGifrqo5LOr2GQ4agpI'], got ['call_tKIA0SvrpV0YwVmAl83zWGZ8']", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [49]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: 두 가지 질문으로 퀴즈를 만들어줘. 하나는 주관식, 하나는 객관식으로, 그런 다음 답변에 대한 피드백을 보내줘

